In [1]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cPickle as pickle
import scipy.io
import time
import ssn
import ks_test3
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials

%matplotlib inline

In [ ]:
# Define Hyperopt search space:
space = [hp.uniform('sig_EE',7,9), 
         hp.uniform('sig_IE',10,16), 
         hp.uniform('sig_EI',3,5),
         hp.uniform('sig_II',3,5),
         hp.uniform('J_EI',0.089,0.105),
         hp.uniform('J_II',0.08,0.105)]

In [9]:
# load Blasdel orientation and ocular dominance maps (previously processed,
# see map_analysis.ipynb
st = time.time()
[OD_map_full, OP_map_full] = np.load('saved_vars/maps-Nov-7.p', 'rb')
print "Elapsed time to load maps: %d seconds" % (time.time() - st)

# plt.figure()
# plt.imshow(OD_map_full)
# plt.colorbar()
# plt.title('Full ocular dominance map, Obermayer and Blasdel')

# plt.figure()
# plt.imshow(OP_map_full)
# plt.colorbar()
# plt.title('Full orientation map, Obermayer and Blasdel')

OD_map = OD_map_full[-75:,-75:]
OP_map = np.floor(OP_map_full[-75:,-75:])

Elapsed time to load maps: 0 seconds
(139, 187)
21.3846153846
24.9333333333
180.196294038


In [ ]:
n_units = 50
selected_units = np.floor( ss_net.N_pairs*np.random.rand(n_units, 2) )

OD_prefs = np.zeros(len(selected_units))
for i in range(len(selected_units)):
    xi = selected_units[i,0]
    yi = selected_units[i,1]
    OD_prefs[i] = OD_map[yi,xi]

In [ ]:
# Define objective funtion for hyperopt:
def iot_ssn_ks2d(args):
    sig_EE, sig_IE, sig_EI, sig_II, J_EI, J_II = args
    
    # Generate SSN with specified hyperparams:
    ss_net = ssn.SSNetwork(sig_EE, sig_IE, sig_EI, sig_II, J_EE=0.1, J_IE=0., J_EI, J_II, OP_map=OP_map, OD_map=OD_map)
    
    # TODO: Check the stability of the network and abort if unstable (return high value)

    c = 40
    dt = 0.005
    timesteps = 100
    dx = ss_net.dx
    N_pairs = ss_net.N_pairs
    
    # first find the summation field size (optimal CRF stimulus) for each unit (both E and I)
    stim_sizes = np.linspace(0.75, 2, 5)
    crf_bank = np.zeros( (n_units, 2, len(stim_sizes), N_pairs, N_pairs) )

    for i in range(n_units):
        xi = selected_units[i,0]
        yi = selected_units[i,1]
        ocularity = np.round( OD_map[yi,xi] )
        ori = OP_map[yi,xi]
        for j in range(len(stim_sizes)):
            crf_bank[i,0,j,:,:] = ssn.generate_mono_stimulus( ori, stim_sizes[j], [dx*xi, dx*yi], OP_map )
            crf_bank[i,1,j,:,:] = ssn.generate_ext_stimulus( ori, stim_sizes[j], [dx*xi, dx*yi], OP_map, OD_map, ocularity)
    
    # Store the summation field sizes (SFS) for both E and I units
    sfs_E = np.zeros( n_units )
    sfs_I = np.copy(sfs_E)
    max_fr_E = np.copy(sfs_E)
    max_fr_I = np.copy(sfs_E)
    
    
    # run to find monocular SFS:
    for i in range(n_units):
        xi = selected_units[i,0]
        yi = selected_units[i,1]
        e_found = False
        i_found = False
        for j in range(len(stim_sizes)):

            if e_found == True and i_found == True:
                break

            h = crf_bank[i,1,j,:,:]
            [r_E, r_I, I_E, I_I] = ss_net.run_simulation(dt, timesteps, c, h )

            if r_E[-1,yi,xi] >= max_fr_E[i]:
                max_fr_E[i] = r_E[-1,yi,xi]
                sfs_E[i] = stim_sizes[j]
            else:
                e_found = True

            if r_I[-1,yi,xi] >= max_fr_I[i]:
                max_fr_I[i] = r_I[-1,yi,xi]
                sfs_I[i] = stim_sizes[j]
            else:
                i_found = True
    
    # Generate non-dominant CRF stimuli
    non_dom_stimuli = np.zeros((len(selected_units), 2, N_pairs, N_pairs))
    for i in range(len(selected_units)):
        xi = selected_units[i,0]
        yi = selected_units[i,1]
        ocularity = np.abs( np.round(OD_prefs[i]) - 1)
        non_dom_stimuli[i,0,:,:] = ssn.generate_ext_stimulus( ori, sfs_E[i], [dx*xi, dx*yi], OP_map, OD_map, ocularity)
        if sfs_E[i] != sfs_I[i]:
            non_dom_stimuli[i,1,:,:] = ssn.generate_ext_stimulus( ori, sfs_I[i], [dx*xi, dx*yi], OP_map, OD_map, ocularity)
    
    non_dom_results = np.zeros((len(selected_units), 2))

    for i in range(len(selected_units)):
        xi = selected_units[i,0]
        yi = selected_units[i,1]
        h = non_dom_stimuli[i,0,:,:]
        [r_E, r_I, I_E, I_I] = ss_net.run_simulation(dt, timesteps, c, h )
        non_dom_results[i,0] = r_E[-1,yi,xi]
        non_dom_results[i,1] = r_I[-1,yi,xi]
        if sfs_E[i] != sfs_I[i]:
            h = non_dom_stimuli[i,1,:,:]
            [r_E, r_I, I_E, I_I] = ss_net.run_simulation(dt, timesteps, c, h )
            non_dom_results[i,1] = r_I[-1,yi,xi]

    threshold = 1 # threshold for Webb's "reliable response" criterion
    # Only carry on with units whose non-dom CRF response is above the threshold:
    thresh_units_E = selected_units[np.where(non_dom_results[:,0]>=threshold),:][0]
    thresh_units_I = selected_units[np.where(non_dom_results[:,1]>=threshold),:][0]

    thresh_units_sfs_E = sfs_E[np.where(non_dom_results[:,0]>=threshold)]
    thresh_units_sfs_I = sfs_I[np.where(non_dom_results[:,1]>=threshold)]

    thresh_units_max_fr_E = max_fr_E[np.where(non_dom_results[:,0]>=threshold)]
    thresh_units_max_fr_I = max_fr_I[np.where(non_dom_results[:,1]>=threshold)]
    
    # Now find which units which are above threshold also suppress below 90% with non-dom surround:
    non_dom_surround_stim_E = np.zeros((len(thresh_units_E), N_pairs, N_pairs))
    dom_surround_stim_E = np.copy(non_dom_surround_stim_E)
    dom_crf_stim_E = np.copy(non_dom_surround_stim_E)
    for i in range(len(thresh_units_E)):
        xi = thresh_units_E[i,0]
        yi = thresh_units_E[i,1]
        inner_d = thresh_units_sfs_E[i]
        outer_d = inner_d + 3
        centre = [dx*xi, dx*yi]
        ocularity = np.abs( np.round(OD_map[yi,xi]) - 1)
        non_dom_surround_stim_E[i] = ssn.generate_ring_stimulus(OP_map[yi,xi], inner_d, outer_d, centre, ocularity, OP_map, OD_map)
        dom_surround_stim_E[i] = ssn.generate_ring_stimulus(OP_map[yi,xi], inner_d, outer_d, centre, np.round(OD_map[yi,xi]), OP_map, OD_map)
        dom_crf_stim_E[i] = ssn.generate_ext_stimulus( ori, inner_d, [dx*xi, dx*yi], OP_map, OD_map, np.round(OD_map[yi,xi]) )
            
    # Run simulations to analyze non dominant suppression:
    non_dom_surround_results = np.zeros((len(thresh_units_E)))
    dom_surround_results = np.copy(non_dom_surround_results)
    for i in range(len(thresh_units_E)):
        xi = thresh_units_E[i,0]
        yi = thresh_units_E[i,1]
        h = non_dom_surround_stim_E[i] + dom_crf_stim_E[i]
        [r_E, r_I, I_E, I_I] = ss_net.run_simulation(dt, timesteps, c, h )
        non_dom_surround_results[i] = r_E[-1,yi,xi]

        h = dom_surround_stim_E[i] + dom_crf_stim_E[i]
        [r_E, r_I, I_E, I_I] = ss_net.run_simulation(dt, timesteps, c, h )
        dom_surround_results[i] = r_E[-1,yi,xi]
        
    dominant_SI_E = (thresh_units_max_fr_E - dom_surround_results) / thresh_units_max_fr_E
    non_dom_SI_E = (thresh_units_max_fr_E - non_dom_surround_results) / thresh_units_max_fr_E
    
    # Now do all the same stuff for the I units:

    non_dom_surround_stim_I = np.zeros((len(thresh_units_I), N_pairs, N_pairs))
    dom_surround_stim_I = np.copy(non_dom_surround_stim_I)
    dom_crf_stim_I = np.copy(non_dom_surround_stim_I)
    for i in range(len(thresh_units_I)):
        xi = thresh_units_I[i,0]
        yi = thresh_units_I[i,1]
        inner_d = thresh_units_sfs_I[i]
        outer_d = inner_d + 3
        centre = [dx*xi, dx*yi]
        ocularity = np.abs( np.round(OD_map[yi,xi]) - 1)
        non_dom_surround_stim_I[i] = ssn.generate_ring_stimulus(OP_map[yi,xi], inner_d, outer_d, centre, ocularity, OP_map, OD_map)
        dom_surround_stim_I[i] = ssn.generate_ring_stimulus(OP_map[yi,xi], inner_d, outer_d, centre, np.round(OD_map[yi,xi]), OP_map, OD_map)
        dom_crf_stim_I[i] = ssn.generate_ext_stimulus( ori, inner_d, [dx*xi, dx*yi], OP_map, OD_map, np.round(OD_map[yi,xi]))

    # Run simulations to analyze non dominant suppression:
    non_dom_surround_results_I = np.zeros((len(thresh_units_I)))
    dom_surround_results_I = np.copy(non_dom_surround_results_I)
    for i in range(len(thresh_units_I)):
        xi = thresh_units_I[i,0]
        yi = thresh_units_I[i,1]
        h = non_dom_surround_stim_I[i] + dom_crf_stim_I[i]
        [r_E, r_I, I_E, I_I] = ss_net.run_simulation(dt, timesteps, c, h )
        non_dom_surround_results_I[i] = r_I[-1,yi,xi]

        h = dom_surround_stim_I[i] + dom_crf_stim_I[i]
        [r_E, r_I, I_E, I_I] = ss_net.run_simulation(dt, timesteps, c, h )
        dom_surround_results_I[i] = r_I[-1,yi,xi]                                                  
    
    dominant_SI_I = (thresh_units_max_fr_I - dom_surround_results_I) / thresh_units_max_fr_I
    non_dom_SI_I = (thresh_units_max_fr_I - non_dom_surround_results_I) / thresh_units_max_fr_I
    
    # Concatenate the E and I results
    model_data_x = np.concatenate((dominant_SI_E, dominant_SI_I))
    model_data_y = np.concatenate((non_dom_SI_E, non_dom_SI_I))

    webb_data = np.array([[0.3538, 0.3214],
    [0.5513, 0.2271],
    [0.5154, 0.5064],
    [0.5641, 0.5681],
    [0.6077, 0.5605],
    [0.7179, 0.6172],
    [0.7487, 0.6865],
    [0.8282, 0.6406],
    [0.8923, 0.5459],
    [0.9282, 0.5690],
    [0.6308, 0.4093],
    [0.7385, 0.4557],
    [0.7923, 0.4866],
    [0.7385, 0.5352],
    [0.9974, 0.9846]])
    
    d, prob = ks_test3.ks2d2s(webb_data[:,0], webb_data[:,1], model_data_x, model_data_y)

    return {
        'status': 'ok',
        'loss':, 1-prob,
        'attachments': {'units_probed':pickle.dumps([thresh_units_E, thresh_units_I, thresh_untits_max_fr_E, thresh_units_max_fr_I, dom_surround_results, dom_surround_results_I, sfs_E, sfs_I])}
        }

In [ ]:
# create a Trials database to store experiment results:
trials = Trials()

st = time.time()
best = fmin(iot_ssn_ks2d, space, algo=tpe.suggest, max_evals=10, trials=trials)
print "Elapsed time for 10 hyperopt sims: %d seconds." % (time.time()-st)
print 'tpe:', best